In [82]:
import pandas as pd
import numpy as np
import os
import time
import logging

In [2]:
# Configure logger
logging.basicConfig(
    level = logging.INFO,
    format = "%(asctime)s - %(levelname)s - %(message)s",
    handlers = [
        logging.StreamHandler(),
        logging.FileHandler("pipeline.log", mode = "a")
    ]
)
logger = logging.getLogger(__name__)

In [147]:
def data_wrangler(
    path_to_dataset, 
    drop_threshold = 0.7, 
    impute_categoricals = True, 
    impute_numericals="median", 
    feature_engineering = True, 
    rare_provider_threshold = 500
):
    """
    Load, clean, and preprocess raw tabular datasets into a standardized pandas DataFrame.

    This function acts as an entry-point for data ingestion and wrangling.
    It automatically detects the dataset type from the file extension 
    (CSV, Excel, Parquet, JSON) and applies consistent preprocessing steps 
    to ensure the data is ready for downstream modeling.

    The wrangling process includes:
        - Loading the dataset from supported formats (.csv, .xlsx, .xls, .parquet, .json).
        - Handle duplicates
        - Handling missing values:
            * Drop features with >= drop_threshold missingness
            * Optionally impute categorical features with 'missing' and '-1' for card features

    Parameters
    ----------
    path_to_dataset : str
        Path to the dataset file (CSV, Excel, Parquet, or JSON).
    drop_threshold : float, default=0.7
        Threshold (0–1) for dropping columns with high missingness.
    impute_categoricals : bool, default=True
        If True, imputes categorical missing values with "missing".
    impute_numericals : str, optional
        Strategy for imputing numerical values. Options: 'mean', 'median'. Default is 'median'.
    feature_engineering : bool, optional
        Whether to engineer new fraud-related features. Default is True.
    rare_provider_threshold : int, optional
        Frequency threshold below which providers are grouped into "rare". Default is 500.

    Returns
    -------
    pandas.DataFrame
        A cleaned and preprocessed DataFrame ready for exploratory data analysis (EDA) 
        or model training.

    Raises
    ------
    FileNotFoundError
        If the provided file path does not exist.
    ValueError
        If the file extension is unsupported.
    RuntimeError
        If there is an error during file loading or preprocessing.
    """

    # load dataset
    start_time = time.perf_counter()
    
    if not os.path.exists(path_to_dataset):
        raise FileNotFoundError(f"File not found: {path_to_dataset}")

    ext = os.path.splitext(path_to_dataset)[1].lower()

    loaders = {
        ".csv": pd.read_csv,
        ".xlsx": pd.read_excel,
        ".xls": pd.read_excel,
        ".parquet": pd.read_parquet,
        ".json": pd.read_json,
    }

    if ext not in loaders:
        raise ValueError(f"Unsupported file type: {ext}. Supported types: {list(loaders.keys())}")

    try:
        # load dataset
        dataset = loaders[ext](path_to_dataset)
        logger.info(f"[OK] Loaded {path_to_dataset} | Shape: {dataset.shape} | Load time: {(time.perf_counter() - start_time):.2f} sec")

        # check for duplicates and fix them
        if dataset.shape[0] == dataset['TransactionID'].nunique():
            print('No duplicates found in dataset')
        else:
            print(f"{dataset['TransactionID'].duplicated().sum()} duplicates found!")
            dataset = dataset.drop_duplicates(subset=['TransactionID'], keep='first')

        # check for missing values
        missing = dataset.isna().mean()
        missing = missing[missing > 0].sort_values(ascending = False)

        if not missing.empty:
            logger.info("Features with missing values:")
            for col, val in missing.items():
                logger.info(f"{col}: ({val:.2%}) missing")

            # drop features above threshold
            to_drop = missing[missing >= drop_threshold].index
            if len(to_drop) > 0:
                dataset.drop(columns = to_drop,axis = 1, inplace = True)
                logger.info(f"Dropped {len(to_drop)} feature(s) due to >={drop_threshold:.0%} missing values: {list(to_drop)}")

            # impute categorical variables if enabled
            if impute_categoricals:
                categorical_cols = dataset.select_dtypes(include=['object', 'category']).columns
                for col in categorical_cols:
                    if dataset[col].isna().any():
                        dataset[col].fillna('missing', inplace = True)
                        logger.info(f"Imputed missing values in categorical feature '{col}' with 'missing'")
                        
                # impute missing values in card and address features with -1
                for col in ['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2']:
                    if dataset[col].isna().any():
                        dataset[col].fillna(-1, inplace = True)
                        logger.info(f"Imputed missing values in card categorical feature '{col}' with '-1'")

            # imput numerical
            if impute_numericals is not None:
                num_cols = dataset.select_dtypes(include = 'number').columns
                for col in num_cols:
                    if dataset[col].isna().sum() > 0:
                        if impute_numericals == 'mean':
                            fill_value = dataset[col].mean()
                        elif impute_numericals == 'median':
                            fill_value = dataset[col].median()
                        else:
                            raise ValueError("Invalid value for impute_numericals. Use 'mean', 'median', or None.")
                        dataset[col] = dataset[col].fillna(fill_value)
                        logger.info("Imputed missing numerical values in '%s' with %s", col, impute_numericals)
        else:
            logger.info("No missing values found!")

        # round all amounts to 2 d.p.
        dataset.TransactionAmt = dataset.TransactionAmt.round(2)
        logger.info("Rounded all amounts to 2 d.p.")


        # --------------------------- Feature Engineering --------------------------------
        if feature_engineering:
            logger.info("Starting feature engineering...")
        
            new_features = pd.DataFrame(index=dataset.index)  # placeholder to collect new cols
        
            # --- composite variable from address (billing_address)
            if "addr1" in dataset.columns and "addr2" in dataset.columns:
                new_features["billing_address"] = dataset["addr1"].astype(str) + "_" + dataset["addr2"].astype(str)
                logger.info("Created `billing_address` feature by combining the address features")
        
            # --- email domain features
            if "P_emaildomain" in dataset.columns:
                new_features["provider"] = dataset["P_emaildomain"].apply(
                    lambda x: x.split(".")[0] if x != "missing" else "missing"
                )
                new_features["suffix"] = dataset["P_emaildomain"].apply(
                    lambda x: x.split(".")[-1] if x != "missing" else "missing"
                )
        
                # bucket rare providers
                provider_counts = new_features["provider"].value_counts()
                new_features["provider_bucket"] = new_features["provider"].apply(
                    lambda x: x if provider_counts[x] >= rare_provider_threshold else "rare"
                )
        
            # --- add all engineered features in one shot
            dataset = pd.concat([dataset, new_features], axis=1)
        
            # --- fraud bucket per domain (depends on isFraud column)
            if "P_emaildomain" in dataset.columns and "isFraud" in dataset.columns:
                fraud_stats = dataset.groupby("P_emaildomain")["isFraud"].agg(["mean", "count"]).reset_index()
                fraud_stats["fraud_bucket"] = pd.cut(
                    fraud_stats["mean"],
                    bins=[-1, 0.03, 0.10, 1],
                    labels=["low", "medium", "high"]
                ).astype(str)
        
                dataset = dataset.merge(
                    fraud_stats[["P_emaildomain", "fraud_bucket"]],
                    on="P_emaildomain",
                    how="left"
                )
                logger.info("Fraud bucket feature added.")
                
            dataset = dataset.drop(columns=["P_emaildomain"])
            logger.info("Dropped original `P_emaildomain` feature after engineering.")
            
            logger.info("Feature engineering completed.")


        elapsed = time.perf_counter() - start_time
        logger.info(f"Data wrangling completed | Final shape: {dataset.shape} | Time: {elapsed:.2f}s")
        
        return dataset
    except Exception as e:
        raise RuntimeError(f"Failed to load {path_to_dataset}: {e}")

In [38]:
# --- Fraud rate per domain ---
fraud_stats = df.groupby('P_emaildomain')['isFraud'].agg(['mean','count']).reset_index()
fraud_stats

def fraud_bucket(row):
    if row['count'] < 100:
        return 'rare'
    elif row['mean'] >= 0.10:
        return 'high'
    elif row['mean'] >= 0.03:
        return 'medium'
    else:
        return 'low'

fraud_stats['fraud_bucket'] = fraud_stats.apply(fraud_bucket, axis=1)

False

In [139]:
df.groupby('P_emaildomain')['isFraud'].agg(['mean','count']).reset_index()

,P_emaildomain,mean,count
0,aim.com,0.126984,315
1,anonymous.com,0.023217,36998
2,aol.com,0.021811,28289
3,att.net,0.007439,4033
4,bellsouth.net,0.027763,1909
5,cableone.net,0.018868,159
6,centurylink.net,0.000000,205
7,cfl.rr.com,0.000000,172
8,charter.net,0.030637,816
9,comcast.net,0.031187,7888


In [146]:
df = data_wrangler("dataset/train_transaction.csv")

2025-10-01 00:31:28,695 - INFO - [OK] Loaded dataset/train_transaction.csv | Shape: (590540, 394) | Load time: 10.64 sec


No duplicates found in dataset


2025-10-01 00:31:29,124 - INFO - Features with missing values:
2025-10-01 00:31:29,124 - INFO - dist2: (93.63%) missing
2025-10-01 00:31:29,125 - INFO - D7: (93.41%) missing
2025-10-01 00:31:29,125 - INFO - D13: (89.51%) missing
2025-10-01 00:31:29,125 - INFO - D14: (89.47%) missing
2025-10-01 00:31:29,126 - INFO - D12: (89.04%) missing
2025-10-01 00:31:29,128 - INFO - D6: (87.61%) missing
2025-10-01 00:31:29,128 - INFO - D8: (87.31%) missing
2025-10-01 00:31:29,129 - INFO - D9: (87.31%) missing
2025-10-01 00:31:29,130 - INFO - V156: (86.12%) missing
2025-10-01 00:31:29,130 - INFO - V139: (86.12%) missing
2025-10-01 00:31:29,130 - INFO - V138: (86.12%) missing
2025-10-01 00:31:29,131 - INFO - V141: (86.12%) missing
2025-10-01 00:31:29,132 - INFO - V146: (86.12%) missing
2025-10-01 00:31:29,132 - INFO - V147: (86.12%) missing
2025-10-01 00:31:29,133 - INFO - V142: (86.12%) missing
2025-10-01 00:31:29,133 - INFO - V149: (86.12%) missing
2025-10-01 00:31:29,134 - INFO - V140: (86.12%) mis

In [148]:
df.isna().sum()

TransactionID      0
isFraud            0
TransactionDT      0
TransactionAmt     0
ProductCD          0
                  ..
billing_address    0
provider           0
suffix             0
provider_bucket    0
fraud_bucket       0
Length: 231, dtype: int64

In [118]:
df['P_emaildomain'].apply(lambda x: x.split('.')[0] if x != 'missing' else 'missing')

0           gmail
1             aol
2         hotmail
3           gmail
4           gmail
           ...   
506686      gmail
506687    hotmail
506688    hotmail
506689    hotmail
506690    hotmail
Name: P_emaildomain, Length: 506691, dtype: object

In [ ]:
# ---Split provider and suffix ---
df['provider'] = df['P_emaildomain'].apply(lambda x: x.split('.')[0] if x != 'missing' else 'missing')
df['suffix'] = df['P_emaildomain'].apply(lambda x: x.split('.')[-1] if x != 'missing' else 'missing')

df[['P_emaildomain', 'provider', 'suffix']].sample(10)

In [108]:
df.P_emaildomain.head()

0      gmail.com
1        aol.com
2    hotmail.com
3      gmail.com
4      gmail.com
Name: P_emaildomain, dtype: object

In [95]:
df['addr2'].unique()

array([ 87.,  -1.,  12.,  96.,  57.,  60.,  65.,  95.,  69.,  73.,  43.,
        68.,  27.,  34.,  88.,  31.,  17.,  38.,  93.,  32.,  13.,  80.,
        62.,  24.,  44.,  64.,  41.,  98., 100.,  78.,  79.,  66.,  61.,
        56.,  30.,  26.,  86.,  52.,  90.,  16., 102.,  67.,  71.,  29.,
        94.,  72.,  28.,  85.,  37.,  59.,  54.,  84.,  45.,  77., 101.,
        19.,  40.,  53.,  48.,  36.,  91.,  33.,  51.,  81.,  47.,  74.,
        99.,  42.,  58.,  11.,  10.,  18.,  46.])

In [81]:
data_wrangler?

Signature: data_wrangler(path_to_dataset, drop_threshold=0.7)
Docstring:
Load, clean, and preprocess raw tabular datasets into a standardized pandas DataFrame.

This function acts as an entry-point for data ingestion and wrangling.
It automatically detects the dataset type from the file extension 
(CSV, Excel, Parquet, JSON) and applies consistent preprocessing steps 
to ensure the data is ready for downstream modeling.

The wrangling process includes:
    - Loading the dataset from supported formats (.csv, .xlsx, .xls, .parquet, .json).
    - Handle duplicates
    - Handling missing values (drop features with >= 'drop_threshold%' missingness or impute otherwise)

Parameters
----------
path_to_dataset : str
    Path to the dataset file (CSV, Excel, Parquet, or JSON).

Returns
-------
pandas.DataFrame
    A cleaned and preprocessed DataFrame ready for exploratory data analysis (EDA) 
    or model training.

Raises
------
FileNotFoundError
    If the provided file path does not exist.
Va

In [83]:
df = pd.read_csv("dataset/train_transaction.csv")

In [85]:
df.isFraud.value_counts(normalize = True)

isFraud
0    0.96501
1    0.03499
Name: proportion, dtype: float64

In [5]:
start_time = time.perf_counter()
df.TransactionID.duplicated().sum()
print(time.perf_counter() - start_time)

0.008370700001250952


In [6]:
start_time = time.perf_counter()
df.duplicated().sum()
print(time.perf_counter() - start_time)

1.0233487999648787


In [40]:
int(df.shape[0] == df.TransactionID.duplicated().sum())

0

In [10]:
start_time = time.perf_counter()
df.drop_duplicates(subset = ['TransactionID'], keep = 'first')
print(time.perf_counter() - start_time)

0.14160339999943972


In [11]:
start_time = time.perf_counter()
df.drop_duplicates()
print(time.perf_counter() - start_time)

1.1088416000129655


In [9]:
df.drop_duplicates(subset = ['TransactionID'], keep = 'first')

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31_missing,id_32_missing,id_33_missing,id_34_missing,id_35_missing,id_36_missing,id_37_missing,id_38_missing,DeviceType_missing,DeviceInfo_missing
0,2987000,0,86400,68.50,W,13926,missing,150.0,discover,142.0,...,1,1,1,1,1,1,1,1,1,1
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,1,1,1,1,1,1,1,1,1,1
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,1,1,1,1,1,1,1,1,1,1
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,1,1,1,1,1,1,1,1,1,1
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,W,6550,missing,150.0,visa,226.0,...,1,1,1,1,1,1,1,1,1,1
590536,3577536,0,15811049,39.50,W,10444,225.0,150.0,mastercard,224.0,...,1,1,1,1,1,1,1,1,1,1
590537,3577537,0,15811079,30.95,W,12037,595.0,150.0,mastercard,224.0,...,1,1,1,1,1,1,1,1,1,1
590538,3577538,0,15811088,117.00,W,7826,481.0,150.0,mastercard,224.0,...,1,1,1,1,1,1,1,1,1,1


In [69]:
df['addr1'].isna().any()

np.True_

In [24]:
ms = df.isna().sum()

In [27]:
ms[ms > 0].sort_values(ascending=False)

dist2    470255
D7       446558
D12      437437
D9       432353
D8       432353
          ...  
C2            3
V293          3
V294          3
V292          3
V295          3
Length: 345, dtype: int64

In [35]:
if not ms.empty:
    print("Features with missing values:")
    for col, val in ms.items():
        print(f"{col}: {val} missing ({val / len(df):.2%})")
else:
    print("No missing values found!")

Features with missing values:
TransactionID: 0 missing (0.00%)
TransactionDT: 0 missing (0.00%)
TransactionAmt: 0 missing (0.00%)
ProductCD: 0 missing (0.00%)
card1: 0 missing (0.00%)
card2: 8654 missing (1.47%)
card3: 3002 missing (0.51%)
card4: 3086 missing (0.52%)
card5: 4547 missing (0.77%)
card6: 3007 missing (0.51%)
addr1: 65609 missing (11.11%)
addr2: 65609 missing (11.11%)
dist1: 291217 missing (49.31%)
dist2: 470255 missing (79.63%)
P_emaildomain: 69192 missing (11.72%)
R_emaildomain: 370821 missing (62.79%)
C1: 3 missing (0.00%)
C2: 3 missing (0.00%)
C3: 3 missing (0.00%)
C4: 3 missing (0.00%)
C5: 3 missing (0.00%)
C6: 3 missing (0.00%)
C7: 3 missing (0.00%)
C8: 3 missing (0.00%)
C9: 3 missing (0.00%)
C10: 3 missing (0.00%)
C11: 3 missing (0.00%)
C12: 3 missing (0.00%)
C13: 4748 missing (0.80%)
C14: 3 missing (0.00%)
D1: 6031 missing (1.02%)
D2: 234769 missing (39.75%)
D3: 203142 missing (34.40%)
D4: 76851 missing (13.01%)
D5: 224375 missing (37.99%)
D6: 381908 missing (64.67

In [51]:
df.isna().mean()

TransactionID     0.00000
isFraud           0.00000
TransactionDT     0.00000
TransactionAmt    0.00000
ProductCD         0.00000
                   ...   
V335              0.86055
V336              0.86055
V337              0.86055
V338              0.86055
V339              0.86055
Length: 394, dtype: float64

In [75]:
df[['card1', 'card2', 'card3', 'card4', 'card5', 'card6']].head()

,card1,card2,card3,card4,card5,card6
0,13926,NaN,150.0,discover,142.0,credit
1,2755,404.0,150.0,mastercard,102.0,credit
2,4663,490.0,150.0,visa,166.0,debit
3,18132,567.0,150.0,mastercard,117.0,debit
4,4497,514.0,150.0,mastercard,102.0,credit


# Alt 2

### For Transaction dataset

In [23]:
import os
import time
import pandas as pd
import logging
import numpy as np

logger = logging.getLogger(__name__)

In [3]:
# Function to load dataset
def load_dataset(path_to_dataset):
    """Load dataset from various file types."""
    if not os.path.exists(path_to_dataset):
        raise FileNotFoundError(f"File not found: {path_to_dataset}")

    ext = os.path.splitext(path_to_dataset)[1].lower()
    loaders = {
        ".csv": pd.read_csv,
        ".xlsx": pd.read_excel,
        ".xls": pd.read_excel,
        ".parquet": pd.read_parquet,
        ".json": pd.read_json,
    }

    if ext not in loaders:
        raise ValueError(f"Unsupported file type: {ext}. Supported: {list(loaders.keys())}")

    return loaders[ext](path_to_dataset)

In [4]:
# Function to handle duplicates in transaction datasets
def handle_duplicates(dataset, id_col="TransactionID"):
    """Drop duplicate rows based on an ID column."""
    if id_col in dataset.columns:
        before = dataset.shape[0]
        dataset = dataset.drop_duplicates(subset=[id_col], keep="first")
        dropped = before - dataset.shape[0]
        if dropped > 0:
            logger.info(f"Dropped {dropped} duplicate rows.")
        else:
            logger.info("No duplicates found!")
    return dataset

In [5]:
def handle_transaction_missing(dataset, drop_threshold=0.7, impute_categoricals=True, impute_numericals="median"):
    """Drop and impute missing values based on rules."""
    missing = dataset.isna().mean()
    to_drop = missing[missing >= drop_threshold].index

    if len(to_drop) > 0:
        dataset.drop(columns=to_drop, inplace=True)
        logger.info(f"Dropped {len(to_drop)} feature(s) with >={drop_threshold:.0%} missingness.")

    # impute categoricals
    if impute_categoricals:
        categorical_cols = dataset.select_dtypes(include=["object", "category"]).columns
        for col in categorical_cols:
            if dataset[col].isna().any():
                dataset[col].fillna("missing", inplace=True)

        for col in ["card1", "card2", "card3", "card5", "addr1", "addr2"]:
            if col in dataset.columns and dataset[col].isna().any():
                dataset[col].fillna(-1, inplace=True)

    # impute numericals
    if impute_numericals is not None:
        num_cols = dataset.select_dtypes(include="number").columns
        for col in num_cols:
            if dataset[col].isna().any():
                if impute_numericals == "mean":
                    fill_value = dataset[col].mean()
                elif impute_numericals == "median":
                    fill_value = dataset[col].median()
                else:
                    raise ValueError("Invalid impute_numericals option. Use 'mean', 'median', or None.")
                dataset[col].fillna(fill_value, inplace=True)

    return dataset

In [6]:
# Function for feature engineering
def trans_feature_engineering(dataset, rare_provider_threshold=500):
    """Engineer fraud-related and domain features."""
    start_time = time.perf_counter()
    new_features = pd.DataFrame(index=dataset.index)

    # billing address
    if "addr1" in dataset.columns and "addr2" in dataset.columns:
        new_features["billing_address"] = dataset["addr1"].astype(str) + "_" + dataset["addr2"].astype(str)

    # email provider features
    if "P_emaildomain" in dataset.columns:
        new_features["provider"] = dataset["P_emaildomain"].apply(
            lambda x: x.split(".")[0] if x != "missing" else "missing"
        )
        new_features["suffix"] = dataset["P_emaildomain"].apply(
            lambda x: x.split(".")[-1] if x != "missing" else "missing"
        )

        provider_counts = new_features["provider"].value_counts()
        new_features["provider_bucket"] = new_features["provider"].apply(
            lambda x: x if provider_counts[x] >= rare_provider_threshold else "rare"
        )

    dataset = pd.concat([dataset, new_features], axis=1)

    # fraud bucket per domain
    if "P_emaildomain" in dataset.columns and "isFraud" in dataset.columns:
        fraud_stats = dataset.groupby("P_emaildomain")["isFraud"].agg(["mean", "count"]).reset_index()
        fraud_stats["fraud_bucket"] = pd.cut(
            fraud_stats["mean"], bins=[-1, 0.03, 0.10, 1], labels=["low", "medium", "high"]
        ).astype(str)

        dataset = dataset.merge(
            fraud_stats[["P_emaildomain", "fraud_bucket"]],
            on="P_emaildomain", how="left"
        )

        dataset.drop(columns=["P_emaildomain"], inplace=True)
    logger.info(f"Feature engineering completed. Time taken: {(time.perf_counter() - start_time):.2f}s")
    return dataset

In [13]:
# Main data wrangling function (to assemble all components together)
def transaction_data_wrangler(
    path_to_dataset,
    drop_threshold=0.7,
    impute_categoricals=True,
    impute_numericals="median",
    feature_engineering_flag=True,
    rare_provider_threshold=500
):
    """Main entry point for transaction dataset wrangling."""
    start_time = time.perf_counter()

    try:
        dataset = load_dataset(path_to_dataset)
        logger.info(f"Loaded dataset | Shape: {dataset.shape} | Load time: {(time.perf_counter() - start_time):.2f}s")

        dataset = handle_duplicates(dataset)
        dataset = handle_transaction_missing(dataset, drop_threshold, impute_categoricals, impute_numericals)

        # round amounts
        if "TransactionAmt" in dataset.columns:
            dataset["TransactionAmt"] = dataset["TransactionAmt"].round(2)

        # feature engineering
        if feature_engineering_flag:
            dataset = trans_feature_engineering(dataset, rare_provider_threshold)

        elapsed = time.perf_counter() - start_time
        logger.info(f"Wrangling completed | Shape: {dataset.shape} | Time: {elapsed:.2f}s")
        return dataset

    except Exception as e:
        raise RuntimeError(f"Failed wrangling {path_to_dataset}: {e}")

In [14]:
df = transaction_data_wrangler('dataset/cleaned_fraud_dataset.parquet')

In [15]:
df.isna().sum()

TransactionID         0
isFraud               0
TransactionDT         0
TransactionAmt        0
ProductCD             0
                     ..
id_37_missing         0
id_38_missing         0
DeviceType_missing    0
DeviceInfo_missing    0
billing_address       0
Length: 81, dtype: int64

In [34]:
df.columns.tolist()

['TransactionID',
 'isFraud',
 'TransactionDT',
 'TransactionAmt',
 'ProductCD',
 'card1',
 'card2',
 'card3',
 'card4',
 'card5',
 'card6',
 'addr1',
 'addr2',
 'dist1',
 'dist2',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'D10',
 'D11',
 'D15',
 'billing_address',
 'provider_bucket',
 'fraud_bucket',
 'suffix_bucket',
 'id_12',
 'id_13',
 'id_14',
 'id_15',
 'id_16',
 'id_17',
 'id_18',
 'id_19',
 'id_20',
 'id_28',
 'id_29',
 'id_30',
 'id_31',
 'id_32',
 'id_33',
 'id_34',
 'id_35',
 'id_36',
 'id_37',
 'id_38',
 'DeviceType',
 'DeviceInfo',
 'id_01_missing',
 'id_02_missing',
 'id_03_missing',
 'id_04_missing',
 'id_05_missing',
 'id_06_missing',
 'id_09_missing',
 'id_10_missing',
 'id_11_missing',
 'id_12_missing',
 'id_13_missing',
 'id_14_missing',
 'id_15_missing',
 'id_16_missing',
 'id_17_missing',
 'id_18_missing',
 'id_19_missing',
 'id_20_missing',
 'id_28_missing',
 'id_29_missing',
 'id_30_missing',
 'id_31_missing',
 'id_32_missing',
 'id_33_missing',
 'id_34_missing',
 'id_

# For Identity Dataset

In [16]:
path = "dataset/train_identity.csv"
df = load_dataset(path)

In [19]:
df = handle_identity_missing(df)
print(df.isna().sum())

TransactionID    0
id_01            0
id_02            0
id_03            0
id_04            0
id_05            0
id_06            0
id_09            0
id_10            0
id_11            0
id_12            0
id_13            0
id_14            0
id_15            0
id_16            0
id_17            0
id_18            0
id_19            0
id_20            0
id_28            0
id_29            0
id_30            0
id_31            0
id_32            0
id_33            0
id_34            0
id_35            0
id_36            0
id_37            0
id_38            0
DeviceType       0
DeviceInfo       0
dtype: int64


C:\Users\NEW\AppData\Local\Temp\ipykernel_26456\617817227.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[col].fillna("missing", inplace=True)
C:\Users\NEW\AppData\Local\Temp\ipykernel_26456\617817227.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, wh

In [18]:
def handle_identity_missing(
    dataset, 
    drop_threshold = 0.7, 
    impute_categoricals = True, 
    impute_numericals = "median"
):
    """Drop and impute missing values based on rules. (Identity dataset)"""
    start_time = time.perf_counter()
    missing = dataset.isna().mean()
    to_drop = missing[missing >= drop_threshold].index

    if len(to_drop) > 0:
        dataset.drop(columns=to_drop, inplace=True)
        logger.info(f"Dropped {len(to_drop)} feature(s) with >={drop_threshold:.0%} missingness.")

    # impute categoricals
    if impute_categoricals:
        categorical_cols = dataset.select_dtypes(include=["object", "category"]).columns
        for col in categorical_cols:
            if dataset[col].isna().any():
                dataset[col].fillna("missing", inplace=True)
        logger.info("Imputed categorical missing values")

    # impute numericals
    if impute_numericals is not None:
        num_cols = dataset.select_dtypes(include="number").columns
        for col in num_cols:
            if dataset[col].isna().any():
                if impute_numericals == "mean":
                    fill_value = dataset[col].mean()
                elif impute_numericals == "median":
                    fill_value = dataset[col].median()
                else:
                    raise ValueError("Invalid impute_numericals option. Use 'mean', 'median', or None.")
                dataset[col].fillna(fill_value, inplace = True)
        logger.info("Imputed numerical values")
    logger.info(f"Data cleaning completed | Time taken: {(time.perf_counter() - start_time):.2f}s")
    return dataset

In [22]:
# Function for feature engineering
def identity_feature_engineering(dataset, rare_device_info_threshold=200):
    """
    Feature engineering for identity dataset:
      - Bucket rare DeviceInfo values into 'others'
      - Regroup/simplify DeviceInfo categories into broader families

    Parameters
    ----------
    dataset : pd.DataFrame
        The identity dataset.
    rare_device_info_threshold : int, default=200
        Minimum frequency for a DeviceInfo to be kept. 
        Rare categories are replaced with 'others'.

    Returns
    -------
    pd.DataFrame
        Dataset with engineered DeviceInfo.
    """
    
    start_time = time.perf_counter()
    df = dataset.copy()

    # --- Step 1: Handle rare DeviceInfo ---
    device_counts = df['DeviceInfo'].value_counts()
    frequent_devices = device_counts[device_counts >= rare_device_info_threshold].index
    df['DeviceInfo'] = df['DeviceInfo'].where(df['DeviceInfo'].isin(frequent_devices), 'others')

    logger.info(
        f"DeviceInfo engineered: {len(frequent_devices)} frequent categories kept, "
        f"others bucketed into 'others'."
    )
    # --- Step 2: Vectorized regrouping ---
    device_series = df['DeviceInfo'].astype(str)

    conditions = [
        device_series.str.startswith('rv:'),                         # Firefox
        device_series.str.startswith('SM-'),                         # Samsung Galaxy
        device_series.str.startswith('Moto'),                        # Motorola
        device_series.str.contains('Trident/7.0|rv:11.0', regex=True), # IE 11
        device_series.str.contains('Windows', regex=False),          # Windows
        device_series.str.contains('iOS', regex=False),              # iOS Device
        device_series.str.contains('MacOS', regex=False),            # MacOS
        device_series.eq('SAMSUNG'),                                 # Exact Samsung string
    ]
    choices = [
        'Firefox',
        'Samsung',
        'Motorola',
        'Internet Explorer',
        'Windows',
        'iOS Device',
        'MacOS',
        'Samsung'
    ]

    df['DeviceInfo'] = np.select(conditions, choices, default=device_series)

    logger.info(f"DeviceInfo categories simplified. Unique categories: {dataset['DeviceInfo'].nunique()}")
    logger.info(f"Feature Engineering completed. | Time taken: {(time.perf_counter() - start_time):.2f}s")
    return df

In [24]:
# Main data wrangling function
d = identity_feature_engineering(load_dataset('dataset/train_identity.csv'))
d.DeviceInfo.unique()

array(['others', 'iOS Device', 'Windows', 'MacOS', 'Samsung', 'Firefox',
       'Internet Explorer', 'ALE-L23 Build/HuaweiALE-L23', 'Motorola'],
      dtype=object)

In [25]:
# Main data wrangling function (to assemble all components together)
def identity_data_wrangler(
    path_to_dataset,
    drop_threshold=0.7,
    impute_categoricals=True,
    impute_numericals="median",
    feature_engineering_flag=True,
    rare_device_info_threshold=500
):
    """Main entry point for identity dataset wrangling."""
    start_time = time.perf_counter()

    try:
        dataset = load_dataset(path_to_dataset)
        logger.info(f"Loaded dataset | Shape: {dataset.shape} | Load time: {(time.perf_counter() - start_time):.2f}s")

        dataset = handle_duplicates(dataset)
        dataset = handle_identity_missing(dataset, drop_threshold, impute_categoricals, impute_numericals)

        # feature engineering
        if feature_engineering_flag:
            dataset = identity_feature_engineering(dataset, rare_device_info_threshold)

        elapsed = time.perf_counter() - start_time
        logger.info(f"Wrangling completed | Shape: {dataset.shape} | Time: {elapsed:.2f}s")
        return dataset

    except Exception as e:
        raise RuntimeError(f"Failed wrangling {path_to_dataset}: {e}")

In [26]:
df = identity_data_wrangler(path)
df.DeviceInfo.unique()

C:\Users\NEW\AppData\Local\Temp\ipykernel_26456\617817227.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset[col].fillna("missing", inplace=True)
C:\Users\NEW\AppData\Local\Temp\ipykernel_26456\617817227.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, wh

array(['others', 'iOS Device', 'Windows', 'missing', 'MacOS', 'Samsung',
       'Internet Explorer', 'Firefox'], dtype=object)

In [1]:
import pandas as pd
import numpy as np
from scripts.data_loading import load_dataset
from scripts.modeling.data_prep import prepare_data
from scripts.wranglers import FraudDataWrangler, FraudDatasetMerger
from scripts.modeling.orchestrator import Orchestrator

C:\Users\NEW\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transaction_df = load_dataset("dataset/train_transaction.csv")
identity_df = load_dataset("dataset/train_identity.csv")

In [5]:
# Handle preprocessing
transaction_df = FraudDataWrangler(dataset_type = "transaction").fit_transform(transaction_df)
identity_df = FraudDataWrangler(dataset_type = "identity").fit_transform(identity_df)

2025-10-08 18:52:50,374 - INFO - No duplicates found!
2025-10-08 18:52:50,964 - INFO - Dropped 168 feature(s) with >=70% missingness.
2025-10-08 18:52:57,106 - INFO - Feature engineering completed. Time taken: 2.54s
2025-10-08 18:52:57,188 - INFO - [transaction] transform() complete | Shape: (590540, 230) | Time: 7.18s
2025-10-08 18:52:57,211 - INFO - No duplicates found!
2025-10-08 18:52:57,284 - INFO - Dropped 9 feature(s) with >=70% missingness.
2025-10-08 18:52:57,416 - INFO - Imputed categorical missing values
2025-10-08 18:52:57,457 - INFO - Imputed numerical values
2025-10-08 18:52:57,458 - INFO - Data cleaning completed | Time taken: 0.25s
2025-10-08 18:52:57,507 - INFO - DeviceInfo engineered: 24 frequent categories kept, others bucketed into 'others'.
2025-10-08 18:52:57,629 - INFO - DeviceInfo categories simplified. Unique categories: 10
2025-10-08 18:52:57,630 - INFO - Feature Engineering completed. | Time taken: 0.17s
2025-10-08 18:52:57,637 - INFO - [identity] transform()

In [13]:
target_cols = [f"id_{i}" for i in range(12, 39)]
existing_cols = [col for col in target_cols if col in identity_df.columns]

print(existing_cols)

['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']


In [24]:
transaction_df.select_dtypes(include=['object', 'category'])

,ProductCD,card4,card6,M1,M2,M3,M4,M5,M6,M7,M8,M9,billing_address,provider,suffix,provider_bucket,fraud_bucket
0,W,discover,credit,T,T,T,M2,F,T,missing,missing,missing,315.0_87.0,missing,missing,missing,low
1,W,mastercard,credit,missing,missing,missing,M0,T,T,missing,missing,missing,325.0_87.0,gmail,com,gmail,medium
2,W,visa,debit,T,T,T,M0,F,F,F,F,F,330.0_87.0,outlook,com,outlook,medium
3,W,mastercard,debit,missing,missing,missing,M0,T,F,missing,missing,missing,476.0_87.0,yahoo,com,yahoo,low
4,H,mastercard,credit,missing,missing,missing,missing,missing,missing,missing,missing,missing,420.0_87.0,gmail,com,gmail,medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,W,visa,debit,T,T,T,M0,T,F,F,F,T,272.0_87.0,missing,missing,missing,low
590536,W,mastercard,debit,T,F,F,M0,F,T,F,F,F,204.0_87.0,gmail,com,gmail,medium
590537,W,mastercard,debit,T,F,F,missing,missing,T,missing,missing,missing,231.0_87.0,gmail,com,gmail,medium
590538,W,mastercard,debit,T,T,T,M0,F,T,missing,missing,missing,387.0_87.0,aol,com,aol,low


In [21]:
identity_df.select_dtypes(include = ['object', 'category'])

,id_12,id_15,id_16,id_28,id_29,id_30,id_31,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,NotFound,New,NotFound,New,NotFound,Android 7.0,samsung browser 6.2,2220x1080,match_status:2,T,F,T,T,mobile,others
1,NotFound,New,NotFound,New,NotFound,iOS 11.1.2,mobile safari 11.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,NotFound,Found,Found,Found,Found,missing,chrome 62.0,missing,missing,F,F,T,T,desktop,Windows
3,NotFound,New,NotFound,New,NotFound,missing,chrome 62.0,missing,missing,F,F,T,T,desktop,missing
4,NotFound,Found,Found,Found,Found,Mac OS X 10_11_6,chrome 62.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,NotFound,Found,Found,Found,Found,missing,chrome 66.0 for android,missing,missing,F,F,T,F,mobile,others
144229,NotFound,New,NotFound,New,NotFound,Android 7.1.1,chrome 55.0 for android,855x480,match_status:2,T,F,T,F,mobile,others
144230,NotFound,New,NotFound,New,NotFound,missing,chrome 65.0 for android,missing,missing,F,F,T,F,mobile,others
144231,NotFound,Found,Found,Found,Found,Mac OS X 10_9_5,chrome 66.0,2560x1600,match_status:2,T,F,T,F,desktop,MacOS


In [20]:
identity_df[existing_cols].select_dtypes(include = ['object', 'category'])

,id_12,id_15,id_16,id_28,id_29,id_30,id_31,id_33,id_34,id_35,id_36,id_37,id_38
0,NotFound,New,NotFound,New,NotFound,Android 7.0,samsung browser 6.2,2220x1080,match_status:2,T,F,T,T
1,NotFound,New,NotFound,New,NotFound,iOS 11.1.2,mobile safari 11.0,1334x750,match_status:1,T,F,F,T
2,NotFound,Found,Found,Found,Found,missing,chrome 62.0,missing,missing,F,F,T,T
3,NotFound,New,NotFound,New,NotFound,missing,chrome 62.0,missing,missing,F,F,T,T
4,NotFound,Found,Found,Found,Found,Mac OS X 10_11_6,chrome 62.0,1280x800,match_status:2,T,F,T,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,NotFound,Found,Found,Found,Found,missing,chrome 66.0 for android,missing,missing,F,F,T,F
144229,NotFound,New,NotFound,New,NotFound,Android 7.1.1,chrome 55.0 for android,855x480,match_status:2,T,F,T,F
144230,NotFound,New,NotFound,New,NotFound,missing,chrome 65.0 for android,missing,missing,F,F,T,F
144231,NotFound,Found,Found,Found,Found,Mac OS X 10_9_5,chrome 66.0,2560x1600,match_status:2,T,F,T,F


In [31]:
target_cols = identity_df.select_dtypes(include=['object', 'category']).columns.tolist()
encoding_map = {}

for col in target_cols:
    unique_values = identity_df[col].unique()
    encoding_map[col] = {val:idx for idx, val in enumerate(unique_values)}

In [34]:
for col, mapping in encoding_map.items():
    identity_df[col] = identity_df[col].map(mapping)

identity_df[target_cols]

,id_12,id_15,id_16,id_28,id_29,id_30,id_31,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,1,1,1,0,0,1,0,0,1
2,0,1,1,1,1,2,2,2,2,1,0,0,0,1,2
3,0,0,0,0,0,2,2,2,2,1,0,0,0,1,3
4,0,1,1,1,1,3,2,3,0,0,0,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,0,1,1,1,1,2,114,2,2,1,0,0,2,0,0
144229,0,0,0,0,0,22,33,28,0,0,0,0,2,0,0
144230,0,0,0,0,0,2,107,2,2,1,0,0,2,0,0
144231,0,1,1,1,1,12,113,11,0,0,0,0,2,1,4


In [ ]:
# Merge datasets
merged_df = FraudDatasetMerger(joint_type = "left").fit_transform((transaction_df, identity_df))

In [36]:
target_cols = transaction_df.select_dtypes(include=['object', 'category']).columns.tolist()
encoding_map = {}

for col in target_cols:
    unique_values = transaction_df[col].unique()
    encoding_map[col] = {val:idx for idx, val in enumerate(unique_values)}

for col, mapping in encoding_map.items():
    transaction_df[col] = transaction_df[col].map(mapping)

transaction_df[target_cols]

,ProductCD,card4,card6,M1,M2,M3,M4,M5,M6,M7,M8,M9,billing_address,provider,suffix,provider_bucket,fraud_bucket
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,1,1,1,1,1,0,0,0,0,1,1,1,1,1
2,0,2,1,0,0,0,1,0,1,1,1,1,2,2,1,2,1
3,0,1,1,1,1,1,1,1,1,0,0,0,3,3,1,3,0
4,1,1,0,1,1,1,2,2,2,0,0,0,4,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,0,2,1,0,0,0,1,1,1,1,1,2,5,0,0,0,0
590536,0,1,1,0,2,2,1,0,0,1,1,1,8,1,1,1,1
590537,0,1,1,0,2,2,2,2,0,0,0,0,26,1,1,1,1
590538,0,1,1,0,0,0,1,0,0,0,0,0,24,8,1,8,0


In [37]:
# for transaction dataset
def trans_feature_engineering(dataset, rare_provider_threshold=500):
    """Engineer fraud-related and domain features."""
    start_time = time.perf_counter()
    new_features = pd.DataFrame(index=dataset.index)

    # billing address
    if "addr1" in dataset.columns and "addr2" in dataset.columns:
        new_features["billing_address"] = dataset["addr1"].astype(str) + "_" + dataset["addr2"].astype(str)

    # email provider features
    if "P_emaildomain" in dataset.columns:
        new_features["provider"] = dataset["P_emaildomain"].apply(
            lambda x: x.split(".")[0] if x != "missing" else "missing"
        )
        new_features["suffix"] = dataset["P_emaildomain"].apply(
            lambda x: x.split(".")[-1] if x != "missing" else "missing"
        )

        provider_counts = new_features["provider"].value_counts()
        new_features["provider_bucket"] = new_features["provider"].apply(
            lambda x: x if provider_counts[x] >= rare_provider_threshold else "rare"
        )

    dataset = pd.concat([dataset, new_features], axis=1)

    # fraud bucket per domain
    if "P_emaildomain" in dataset.columns and "isFraud" in dataset.columns:
        fraud_stats = dataset.groupby("P_emaildomain")["isFraud"].agg(["mean", "count"]).reset_index()
        fraud_stats["fraud_bucket"] = pd.cut(
            fraud_stats["mean"], bins=[-1, 0.03, 0.10, 1], labels=["low", "medium", "high"]
        ).astype(str)

        dataset = dataset.merge(
            fraud_stats[["P_emaildomain", "fraud_bucket"]],
            on="P_emaildomain", how="left"
        )

        dataset.drop(columns=["P_emaildomain"], inplace=True)

    # encode categorical variables
    target_cols = dataset.select_dtypes(include=['object', 'category']).columns.tolist()
    encoding_map = {}
    
    for col in target_cols:
        unique_values = dataset[col].unique()
        encoding_map[col] = {val:idx for idx, val in enumerate(unique_values)}

    for col, mapping in encoding_map.items():
        dataset[col] = dataset[col].map(mapping)
    
    logger.info(f"Feature engineering completed. Time taken: {(time.perf_counter() - start_time):.2f}s")
    return dataset

In [39]:
import time
trans_feature_engineering(transaction_df)

AttributeError: 'DataFrame' object has no attribute 'unique'

In [42]:
import pandas as pd
import numpy as np
import logging
import time

# Configure logger
logging.basicConfig(
    level = logging.INFO,
    format = "%(asctime)s - %(levelname)s - %(message)s",
    handlers = [
        logging.StreamHandler(),
        logging.FileHandler("pipeline.log", mode = "a")
    ]
)
logger = logging.getLogger(__name__)

In [58]:
# Function for feature engineering
def identity_feature_engineering(dataset, rare_device_info_threshold=200):
    """
    Feature engineering for identity dataset:
      - Bucket rare DeviceInfo values into 'others'
      - Regroup/simplify DeviceInfo categories into broader families

    Parameters
    ----------
    dataset : pd.DataFrame
        The identity dataset.
    rare_device_info_threshold : int, default=200
        Minimum frequency for a DeviceInfo to be kept. 
        Rare categories are replaced with 'others'.

    Returns
    -------
    pd.DataFrame
        Dataset with engineered DeviceInfo.
    """
    
    start_time = time.perf_counter()
    df = dataset.copy()

    # --- Step 1: Handle rare DeviceInfo ---
    if 'DeviceInfo' in df.columns:
        device_counts = df['DeviceInfo'].value_counts()
        frequent_devices = device_counts[device_counts >= rare_device_info_threshold].index
        df['DeviceInfo'] = df['DeviceInfo'].where(df['DeviceInfo'].isin(frequent_devices), 'others')
    
        logger.info(
            f"DeviceInfo engineered: {len(frequent_devices)} frequent categories kept, "
            f"others bucketed into 'others'."
        )
        # --- Step 2: Vectorized regrouping ---
        device_series = df['DeviceInfo'].astype(str)
    
        conditions = [
            device_series.str.startswith('rv:'),                         # Firefox
            device_series.str.startswith('SM-'),                         # Samsung Galaxy
            device_series.str.startswith('Moto'),                        # Motorola
            device_series.str.contains('Trident/7.0|rv:11.0', regex=True), # IE 11
            device_series.str.contains('Windows', regex=False),          # Windows
            device_series.str.contains('iOS', regex=False),              # iOS Device
            device_series.str.contains('MacOS', regex=False),            # MacOS
            device_series.eq('SAMSUNG'),                                 # Exact Samsung string
        ]
        choices = [
            'Firefox',
            'Samsung',
            'Motorola',
            'Internet Explorer',
            'Windows',
            'iOS Device',
            'MacOS',
            'Samsung'
        ]
    
        df['DeviceInfo'] = np.select(conditions, choices, default=device_series)

    logger.info(f"DeviceInfo categories simplified. Unique categories: {df['DeviceInfo'].nunique()}")

    # encode categorical variables
    target_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    encoding_map = {}
    
    for col in target_cols:
        unique_values = df[col].unique()
        encoding_map[col] = {val:idx for idx, val in enumerate(unique_values)}

    for col, mapping in encoding_map.items():
        df[col] = df[col].map(mapping)
    
    logger.info(f"Feature Engineering completed. | Time taken: {(time.perf_counter() - start_time):.2f}s")
    return df

In [59]:
identity_feature_engineering(identity_df)

2025-10-09 00:06:50,808 - INFO - DeviceInfo engineered: 10 frequent categories kept, others bucketed into 'others'.
2025-10-09 00:06:50,940 - INFO - DeviceInfo categories simplified. Unique categories: 10
2025-10-09 00:06:50,954 - INFO - Feature Engineering completed. | Time taken: 0.16s


,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,...,0,32.0,0,0,0,0,0,0,0,0
1,2987008,-5.0,98945.0,0.0,0.0,0.0,-5.0,0.0,0.0,100.0,...,1,32.0,1,1,0,0,1,0,0,1
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,...,2,24.0,2,2,1,0,0,0,1,2
3,2987011,-5.0,221832.0,0.0,0.0,0.0,-6.0,0.0,0.0,100.0,...,2,24.0,2,2,1,0,0,0,1,3
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,...,2,24.0,3,0,0,0,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,-15.0,145955.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,...,114,24.0,2,2,1,0,0,2,0,0
144229,3577526,-5.0,172059.0,0.0,0.0,1.0,-5.0,0.0,0.0,100.0,...,33,32.0,28,0,0,0,0,2,0,0
144230,3577529,-20.0,632381.0,0.0,0.0,-1.0,-36.0,0.0,0.0,100.0,...,107,24.0,2,2,1,0,0,2,0,0
144231,3577531,-5.0,55528.0,0.0,0.0,0.0,-7.0,0.0,0.0,100.0,...,113,24.0,11,0,0,0,0,2,1,4


In [45]:
trans_feature_engineering(transaction_df)

AttributeError: 'DataFrame' object has no attribute 'unique'

In [44]:
identity_feature_engineering(identity_df)

2025-10-08 20:40:36,171 - INFO - DeviceInfo engineered: 10 frequent categories kept, others bucketed into 'others'.
2025-10-08 20:40:36,306 - INFO - DeviceInfo categories simplified. Unique categories: 10
2025-10-08 20:40:36,318 - INFO - Feature Engineering completed. | Time taken: 0.16s


,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_09,id_10,id_11,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,...,0,32.0,0,0,0,0,0,0,0,0
1,2987008,-5.0,98945.0,0.0,0.0,0.0,-5.0,0.0,0.0,100.0,...,1,32.0,1,1,0,0,1,0,0,1
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,...,2,24.0,2,2,1,0,0,0,1,2
3,2987011,-5.0,221832.0,0.0,0.0,0.0,-6.0,0.0,0.0,100.0,...,2,24.0,2,2,1,0,0,0,1,3
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,0.0,0.0,100.0,...,2,24.0,3,0,0,0,0,0,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144228,3577521,-15.0,145955.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,...,114,24.0,2,2,1,0,0,2,0,0
144229,3577526,-5.0,172059.0,0.0,0.0,1.0,-5.0,0.0,0.0,100.0,...,33,32.0,28,0,0,0,0,2,0,0
144230,3577529,-20.0,632381.0,0.0,0.0,-1.0,-36.0,0.0,0.0,100.0,...,107,24.0,2,2,1,0,0,2,0,0
144231,3577531,-5.0,55528.0,0.0,0.0,0.0,-7.0,0.0,0.0,100.0,...,113,24.0,11,0,0,0,0,2,1,4


In [46]:
def trans_feature_engineering(dataset: pd.DataFrame, rare_provider_threshold: int = 500):
    """Engineer fraud-related and domain features and factorize categorical columns.

    Notes:
    - Uses pd.factorize for fast, vectorized integer encoding.
    - If duplicate column names exist, keeps the first occurrence (logs a warning).
    - Missing values receive code -1 (pd.factorize default na_sentinel).
    """
    start_time = time.perf_counter()
    new_features = pd.DataFrame(index=dataset.index)

    # billing address
    if "addr1" in dataset.columns and "addr2" in dataset.columns:
        new_features["billing_address"] = dataset["addr1"].astype(str) + "_" + dataset["addr2"].astype(str)

    # email provider features
    if "P_emaildomain" in dataset.columns:
        new_features["provider"] = dataset["P_emaildomain"].apply(
            lambda x: x.split(".")[0] if pd.notna(x) and x != "missing" else "missing"
        )
        new_features["suffix"] = dataset["P_emaildomain"].apply(
            lambda x: x.split(".")[-1] if pd.notna(x) and x != "missing" else "missing"
        )

        provider_counts = new_features["provider"].value_counts()
        new_features["provider_bucket"] = new_features["provider"].apply(
            lambda x: x if provider_counts.get(x, 0) >= rare_provider_threshold else "rare"
        )

    # concat new features in one shot (avoids fragmentation warnings)
    dataset = pd.concat([dataset, new_features], axis=1)

    # fraud bucket per domain (depends on isFraud)
    if "P_emaildomain" in dataset.columns and "isFraud" in dataset.columns:
        fraud_stats = dataset.groupby("P_emaildomain")["isFraud"].agg(["mean", "count"]).reset_index()
        fraud_stats["fraud_bucket"] = pd.cut(
            fraud_stats["mean"], bins=[-1, 0.03, 0.10, 1], labels=["low", "medium", "high"]
        ).astype(str)

        dataset = dataset.merge(
            fraud_stats[["P_emaildomain", "fraud_bucket"]],
            on="P_emaildomain",
            how="left"
        )
        # drop original domain column after merging
        dataset.drop(columns=["P_emaildomain"], inplace=True, errors="ignore")

    # ---- Defensive: handle duplicate column names (keep first occurrence) ----
    if dataset.columns.duplicated().any():
        dup_cols = dataset.columns[dataset.columns.duplicated()].unique().tolist()
        logger.warning("Duplicate column names detected and will be deduplicated (keeping first occurrence): %s", dup_cols)
        # Keep only the first occurrence of duplicate column names
        dataset = dataset.loc[:, ~dataset.columns.duplicated()]

    # ---- Encode / factorize categorical columns (fast & safe) ----
    cat_cols = dataset.select_dtypes(include=["object", "category"]).columns.tolist()
    encoding_map = {}

    for col in cat_cols:
        # ensure we're dealing with a Series
        col_series = dataset[col]
        # pd.factorize returns (codes, uniques); codes use -1 for NaN by default
        codes, uniques = pd.factorize(col_series, na_sentinel=-1)
        dataset[col] = codes  # assign integer codes (dtype=np.int64 with -1 for missing)

        # build a JSON-friendly mapping for this column (uniques excludes NaN)
        # store list of uniques in order (index -> value). Missing is represented by -1.
        encoding_map[col] = {
            "mapping_type": "factorize",
            "uniques": uniques.tolist(),   # uniques[i] -> code i ; missing -> -1
            "missing_code": -1
        }

    elapsed = time.perf_counter() - start_time
    logger.info("Feature engineering completed. Time taken: %.2f s", elapsed)
    # if you want to return encoding_map as well, you can return (dataset, encoding_map)
    return dataset


In [48]:
df = trans_feature_engineering(transaction_df)

2025-10-08 21:08:17,025 - WARNING - Duplicate column names detected and will be deduplicated (keeping first occurrence): ['billing_address']
2025-10-08 21:08:17,201 - INFO - Feature engineering completed. Time taken: 0.73 s


In [51]:
df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V317,V318,V319,V320,V321,billing_address,provider,suffix,provider_bucket,fraud_bucket
0,2987000,0,86400,68.5,0,13926,-1.0,150.0,0,142.0,...,117.0,0.0,0.0,0.0,0.0,0,0,0,0,0
1,2987001,0,86401,29.0,0,2755,404.0,150.0,1,102.0,...,0.0,0.0,0.0,0.0,0.0,1,1,1,1,1
2,2987002,0,86469,59.0,0,4663,490.0,150.0,2,166.0,...,0.0,0.0,0.0,0.0,0.0,2,2,1,2,1
3,2987003,0,86499,50.0,0,18132,567.0,150.0,1,117.0,...,1404.0,790.0,0.0,0.0,0.0,3,3,1,3,0
4,2987004,0,86506,50.0,1,4497,514.0,150.0,1,102.0,...,0.0,0.0,0.0,0.0,0.0,4,1,1,1,1


In [50]:
df.billing_address.unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [56]:
cat_cols = transaction_df.select_dtypes(include=["object", "category"]).columns.tolist()
encoding_map = {}

for col in cat_cols:
    # ensure we're dealing with a Series
    col_series = dataset[col]
    print('*' * 45)
    print(col_series)

In [64]:
def trans_feature_engineering(dataset: pd.DataFrame, rare_provider_threshold: int = 500):
    """Engineer fraud-related and domain features and factorize categorical columns.

    Notes:
    - Uses pd.factorize for fast, vectorized integer encoding.
    - If duplicate column names exist, keeps the first occurrence (logs a warning).
    - Missing values receive code -1 (pd.factorize default na_sentinel).
    """
    start_time = time.perf_counter()
    new_features = pd.DataFrame(index=dataset.index)

    # billing address
    if "addr1" in dataset.columns and "addr2" in dataset.columns:
        new_features["billing_address"] = dataset["addr1"].astype(str) + "_" + dataset["addr2"].astype(str)

    # email provider features
    if "P_emaildomain" in dataset.columns:
        new_features["provider"] = dataset["P_emaildomain"].apply(
            lambda x: x.split(".")[0] if pd.notna(x) and x != "missing" else "missing"
        )
        new_features["suffix"] = dataset["P_emaildomain"].apply(
            lambda x: x.split(".")[-1] if pd.notna(x) and x != "missing" else "missing"
        )

        provider_counts = new_features["provider"].value_counts()
        new_features["provider_bucket"] = new_features["provider"].apply(
            lambda x: x if provider_counts.get(x, 0) >= rare_provider_threshold else "rare"
        )

    # concat new features in one shot (avoids fragmentation warnings)
    dataset = pd.concat([dataset, new_features], axis=1)

    # fraud bucket per domain (depends on isFraud)
    if "P_emaildomain" in dataset.columns and "isFraud" in dataset.columns:
        fraud_stats = dataset.groupby("P_emaildomain")["isFraud"].agg(["mean", "count"]).reset_index()
        fraud_stats["fraud_bucket"] = pd.cut(
            fraud_stats["mean"], bins=[-1, 0.03, 0.10, 1], labels=["low", "medium", "high"]
        ).astype(str)

        dataset = dataset.merge(
            fraud_stats[["P_emaildomain", "fraud_bucket"]],
            on="P_emaildomain",
            how="left"
        )
        # drop original domain column after merging
        dataset.drop(columns=["P_emaildomain"], inplace=True, errors="ignore")

    # ---- Defensive: handle duplicate column names (keep first occurrence) ----
    if dataset.columns.duplicated().any():
        dup_cols = dataset.columns[dataset.columns.duplicated()].unique().tolist()
        logger.warning("Duplicate column names detected and will be deduplicated (keeping first occurrence): %s", dup_cols)
        # Keep only the first occurrence of duplicate column names
        dataset = dataset.loc[:, ~dataset.columns.duplicated()]

    # ---- Encode / factorize categorical columns (fast & safe) ----
    cat_cols = dataset.select_dtypes(include=["object", "category"]).columns.tolist()
    encoding_map = {}

    for col in cat_cols:
        # ensure we're dealing with a Series
        col_series = dataset[col]
        # pd.factorize returns (codes, uniques); codes use -1 for NaN by default
        codes, uniques = pd.factorize(col_series, na_sentinel=-1)
        dataset[col] = codes  # assign integer codes (dtype=np.int64 with -1 for missing)

        # build a JSON-friendly mapping for this column (uniques excludes NaN)
        # store list of uniques in order (index -> value). Missing is represented by -1.
        encoding_map[col] = {
            "mapping_type": "factorize",
            "uniques": uniques.tolist(),   # uniques[i] -> code i ; missing -> -1
            "missing_code": -1
        }

    elapsed = time.perf_counter() - start_time
    logger.info("Feature engineering completed. Time taken: %.2f s", elapsed)
    # if you want to return encoding_map as well, you can return (dataset, encoding_map)
    return dataset

In [65]:
trans_feature_engineering(transaction_df)

2025-10-09 00:12:02,099 - WARNING - Duplicate column names detected and will be deduplicated (keeping first occurrence): ['billing_address']
2025-10-09 00:12:02,284 - INFO - Feature engineering completed. Time taken: 0.72 s


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V317,V318,V319,V320,V321,billing_address,provider,suffix,provider_bucket,fraud_bucket
0,2987000,0,86400,68.50,0,13926,-1.0,150.0,0,142.0,...,117.0,0.0,0.000000,0.000000,0.000000,0,0,0,0,0
1,2987001,0,86401,29.00,0,2755,404.0,150.0,1,102.0,...,0.0,0.0,0.000000,0.000000,0.000000,1,1,1,1,1
2,2987002,0,86469,59.00,0,4663,490.0,150.0,2,166.0,...,0.0,0.0,0.000000,0.000000,0.000000,2,2,1,2,1
3,2987003,0,86499,50.00,0,18132,567.0,150.0,1,117.0,...,1404.0,790.0,0.000000,0.000000,0.000000,3,3,1,3,0
4,2987004,0,86506,50.00,1,4497,514.0,150.0,1,102.0,...,0.0,0.0,0.000000,0.000000,0.000000,4,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.00,0,6550,-1.0,150.0,2,226.0,...,0.0,0.0,0.000000,0.000000,0.000000,5,0,0,0,0
590536,3577536,0,15811049,39.50,0,10444,225.0,150.0,1,224.0,...,0.0,0.0,0.000000,0.000000,0.000000,8,1,1,1,1
590537,3577537,0,15811079,30.95,0,12037,595.0,150.0,1,224.0,...,0.0,0.0,0.000000,0.000000,0.000000,26,1,1,1,1
590538,3577538,0,15811088,117.00,0,7826,481.0,150.0,1,224.0,...,2234.0,0.0,0.000000,0.000000,0.000000,24,8,1,8,0


In [80]:
def encode_categorical_features(dataset: pd.DataFrame) -> pd.DataFrame:
    """
    Encode all categorical (object/category) columns in the dataset using integer factorization.

    Args:
        dataset (pd.DataFrame): Input dataset.

    Returns:
        pd.DataFrame: Dataset with categorical features encoded as integers.
    """
    start = time.perf_counter()

    # handle duplicate columns
    if dataset.columns.duplicated().any():
        dup_cols = dataset.columns[dataset.columns.duplicated()].unique().tolist()
        logger.warning(f"Duplicate columns detected and will be deduplicated: {dup_cols}")
        dataset = dataset.loc[:, ~dataset.columns.duplicated()]

    cat_cols = dataset.select_dtypes(include=["object", "category"]).columns.tolist()
    if not cat_cols:
        logger.info("No categorical feature found for encoding.")
        return dataset
        
    for col in cat_cols:
        col_series = dataset[col]
        codes, uniques = pd.factorize(col_series)
        dataset[col] = codes
        logger.info(f"Encoded `{col}` with {len(uniques)} unique values.")

        
    logger.info(f"Encoding categorical columns completed. | Time taken: {time.perf_counter() - start:.2f}s")
    return dataset

In [66]:
# Function to encode categorical features
def encode_categorical_features(dataset):
    """
    Encode categorical features of given datasets automatically.
    """
    cat_cols = dataset.select_dtypes(include = ["object", "category"]).columns.tolist()
    encoding_map = {}

    for col in cat_cols:
        col_series = dataset[col]

        codes, uniques = pd.factorize(col_series)
        dataset[col] = codes

        encoding_map[col] = {
            "mapping_type": "factorize",
            "uniques": uniques.tolist()
        }

    return dataset

In [76]:
encode_categorical_features(transaction_df)

2025-10-09 00:46:57,586 - INFO - Encoded 0 categorical columns in 0.00s


,TransactionID,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663586,-45.0,280290.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,0,NaN,-1,-1,0,0,0,0,0,0
1,3663588,0.0,3579.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,0,24.0,0,0,1,0,0,1,0,1
2,3663597,-5.0,185210.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,...,1,NaN,-1,-1,0,1,0,0,1,2
3,3663601,-45.0,252944.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,0,NaN,-1,-1,0,0,0,0,0,0
4,3663602,-95.0,328680.0,NaN,NaN,7.0,-33.0,NaN,NaN,NaN,...,0,NaN,-1,-1,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141902,4170230,-20.0,473365.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,126,NaN,-1,-1,0,0,0,0,0,1305
141903,4170233,-5.0,489917.0,0.0,0.0,-4.0,-32.0,NaN,NaN,0.0,...,126,NaN,-1,-1,0,0,0,0,0,1631
141904,4170234,-5.0,110081.0,NaN,NaN,22.0,-31.0,NaN,NaN,NaN,...,5,32.0,4,0,1,0,1,1,0,4
141905,4170236,-45.0,266704.0,NaN,NaN,-3.0,-10.0,NaN,NaN,NaN,...,75,NaN,-1,-1,0,0,0,0,0,172


In [81]:
transaction_df = load_dataset("dataset/test_identity.csv")

encode_categorical_features(transaction_df)

2025-10-09 00:51:00,389 - INFO - Encoded `id-12` with 2 unique values.
2025-10-09 00:51:00,398 - INFO - Encoded `id-15` with 3 unique values.
2025-10-09 00:51:00,405 - INFO - Encoded `id-16` with 2 unique values.
2025-10-09 00:51:00,409 - INFO - Encoded `id-23` with 3 unique values.
2025-10-09 00:51:00,412 - INFO - Encoded `id-27` with 2 unique values.
2025-10-09 00:51:00,420 - INFO - Encoded `id-28` with 2 unique values.
2025-10-09 00:51:00,427 - INFO - Encoded `id-29` with 2 unique values.
2025-10-09 00:51:00,434 - INFO - Encoded `id-30` with 86 unique values.
2025-10-09 00:51:00,442 - INFO - Encoded `id-31` with 135 unique values.
2025-10-09 00:51:00,449 - INFO - Encoded `id-33` with 390 unique values.
2025-10-09 00:51:00,455 - INFO - Encoded `id-34` with 2 unique values.
2025-10-09 00:51:00,460 - INFO - Encoded `id-35` with 2 unique values.
2025-10-09 00:51:00,466 - INFO - Encoded `id-36` with 2 unique values.
2025-10-09 00:51:00,472 - INFO - Encoded `id-37` with 2 unique values.
2

,TransactionID,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663586,-45.0,280290.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,0,NaN,-1,-1,0,0,0,0,0,0
1,3663588,0.0,3579.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,0,24.0,0,0,1,0,0,1,0,1
2,3663597,-5.0,185210.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,...,1,NaN,-1,-1,0,1,0,0,1,2
3,3663601,-45.0,252944.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,0,NaN,-1,-1,0,0,0,0,0,0
4,3663602,-95.0,328680.0,NaN,NaN,7.0,-33.0,NaN,NaN,NaN,...,0,NaN,-1,-1,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141902,4170230,-20.0,473365.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,126,NaN,-1,-1,0,0,0,0,0,1305
141903,4170233,-5.0,489917.0,0.0,0.0,-4.0,-32.0,NaN,NaN,0.0,...,126,NaN,-1,-1,0,0,0,0,0,1631
141904,4170234,-5.0,110081.0,NaN,NaN,22.0,-31.0,NaN,NaN,NaN,...,5,32.0,4,0,1,0,1,1,0,4
141905,4170236,-45.0,266704.0,NaN,NaN,-3.0,-10.0,NaN,NaN,NaN,...,75,NaN,-1,-1,0,0,0,0,0,172
